In [2]:
import sys
import pdb
# sys.path.append("C:\\users\\mccar\\miniconda3\\lib\\site-packages")
# import gymnasium as gym
import gym
from bettermdptools.algorithms.planner import Planner
from bettermdptools.utils.plots import Plots
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# env = gym.make('LunarLander-v2')
# env = gym.make("SpaceInvaders-v4")
# env = gym.make("CartPole-v1")
# env = gym.make("CartPole-v0")
# env = gym.make("MountainCar-v0")
env = gym.make("Blackjack-v1")
# all = gym.envs.registry
# for env, val in all.items():
#     print(env)
# print(len(all))


In [11]:
# Value Iteration
verbose = True
gamma = 0.999
# gamma = 1
epsilon = 1e-2

## Initialize the value function
# print(env.observation_space)
# V = np.zeros(env.observation_space.n)
V = np.zeros((32, 11, 2)) 
rewards_by_iteration = []
steps_by_iteration = []
num_iterations = 0
rewards_by_episode = []
steps_by_episode = []
found_it = False
found_it_iter = 0

while True:
    state = env.reset()[0]
    delta = 0
    num_steps = 0
    total_reward = 0
    # For each episode, perform value iteration until convergence
    while True:
        # Increment the number of steps
        num_steps += 1
        # Choose the best action based on the current value function
        action_values = {}
        for action in range(env.action_space.n):
            action_values[action] = sum([prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[state][action]])
        # pick a random move of the max states
        max_value = max(action_values.values())
        max_states = [state for state, value in action_values.items() if value == max_value]
        action = np.random.choice(max_states)
        # Take the action and observe the next state and reward
        next_state, reward, done, trunc, _ = env.step(action)
        if reward > 0:
            if not found_it:
                found_it = True
                found_it_iter = num_iterations
            print("Found IT!!!!")
        # print(num_steps, next_state, reward, done, trunc)
        # Update the total reward
        total_reward += reward
        # Update the value function
        V[state] = max(sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[state][action]) for action in range(env.action_space.n))
        # Check for convergence
        delta = max(delta, np.abs(total_reward - V[state]))
        if done or trunc:
            break
        # print(state)
        state = next_state
    # Increment the number of iterations
    num_iterations += 1
    rewards_by_episode.append(total_reward)
    steps_by_episode.append(num_steps)

    if verbose:
        print(f"Episode {num_iterations} - Reward: {total_reward}, Steps: {num_steps}")
    # Check for convergence
    if (delta < epsilon and num_iterations > 1000) or num_iterations > 3000:
        break

# moving avgs
reward_moving_avg_10 = np.convolve(rewards_by_episode, np.ones(10)/10, mode='valid')
reward_moving_avg_50 = np.convolve(rewards_by_episode, np.ones(50)/50, mode='valid')
reward_moving_avg_100 = np.convolve(rewards_by_episode, np.ones(100)/100, mode='valid')
reward_moving_avg_500 = np.convolve(rewards_by_episode, np.ones(500)/500, mode='valid')
reward_moving_avg_1000 = np.convolve(rewards_by_episode, np.ones(1000)/1000, mode='valid')
steps_moving_avg_10 = np.convolve(steps_by_episode, np.ones(10)/10, mode='valid')
steps_moving_avg_50 = np.convolve(steps_by_episode, np.ones(50)/50, mode='valid')
steps_moving_avg_100 = np.convolve(steps_by_episode, np.ones(100)/100, mode='valid')
steps_moving_avg_500 = np.convolve(steps_by_episode, np.ones(500)/500, mode='valid')
steps_moving_avg_1000 = np.convolve(steps_by_episode, np.ones(1000)/1000, mode='valid')


# Extract and return the optimal policy
policy = np.zeros(env.observation_space.n, dtype=int)
for state in range(env.observation_space.n):
    action_values = [sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[state][action]) for action in range(env.action_space.n)]
    policy[state] = np.argmax(action_values)

# Plot the convergence
fig, ax1 = plt.subplots(figsize=(10, 6))
color = 'tab:red'
ax1.set_xlabel('Episode')
ax1.set_ylabel('Steps per Episode', color=color)
# ax1.plot(steps_moving_avg_1000, label='Steps (MA-1000)', color=color)
ax1.plot(steps_moving_avg_500, label='Steps (MA-500)', color=color)
ax1.plot(steps_moving_avg_100, label='Steps (MA-100)', color=color, linestyle='dashed')
# ax1.plot(steps_moving_avg_50, label='Steps (MA-50)', color=color, linestyle='dashed')
# ax1.plot(steps_moving_avg_10, label='Steps (MA-10)', color=color, linestyle='dotted')
ax1.tick_params(axis='y', labelcolor=color)
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Reward per Episode', color=color)
# ax2.plot(reward_moving_avg_1000, label='Reward (MA-1000)', color=color)
ax2.plot(reward_moving_avg_500, label='Reward (MA-500)', color=color)
ax2.plot(reward_moving_avg_100, label='Reward (MA-100)', color=color, linestyle='dashed')
# ax2.plot(reward_moving_avg_50, label='Reward (MA-50)', color=color, linestyle='dashed')
# ax2.plot(reward_moving_avg_10, label='Reward (MA-10)', color=color, linestyle='dotted')
ax2.tick_params(axis='y', labelcolor=color)
fig.tight_layout()
plt.title('Convergence of Value Iteration Moving Averages (Steps and Reward per Episode)')
plt.show()

# Print the optimal policy
# print("Optimal Policy:")
# print(policy.reshape((8, 8)))
print("Found it in iteration: ", found_it_iter)

# plot state values
size=(8,8)
Plots.values_heat_map(V, "Frozen Lake\nValue Iteration State Values", size)

# plot policy
fl_actions = {0: "←", 1: "↓", 2: "→", 3: "↑"}
fl_map_size=(8,8)
title="FL Mapped Policy\nArrows represent best action"
val_max, policy_map = Plots.get_policy_map(policy, V, fl_actions, fl_map_size)
Plots.plot_policy(val_max, policy_map, fl_map_size, title)

print(np.array(print_env))

AttributeError: 'BlackjackEnv' object has no attribute 'P'

: 

In [ ]:
# Define State Space

# Define the number of bins for each parameter
num_bins = 6
# Define the ranges for each parameter
position_range = [-1.2, 0.6]
velocity_range = [-0.07, 0.07]

def get_stare_space():
    state_space = []

    # Create bins for each parameter
    for cart_pos in range(num_bins):
        for cart_vel in range(num_bins):
            # Calculate the value for each parameter
            pos_value = position_range[0] + cart_pos * (position_range[1] - position_range[0]) / (num_bins - 1)
            velocity_value = velocity_range[0] + cart_vel * (velocity_range[1] - velocity_range[0]) / (num_bins - 1)

            # Add the discretized state to the state space list
            state_space.append([round(pos_value, 3), round(velocity_value,3)])
    return state_space


# Function to initialize the value function for each state to zero
def initialize_value_function(state_space):
    value_function = {}
    for state in state_space:
        value_function[tuple(state)] = 0
    return value_function

state_space = get_stare_space()
POS_RANGE = [x[0] for x in state_space]
VEL_RANGE = [x[1] for x in state_space]

def get_state_bin(state):
    # get the ranges of each of the parameters divided by bins
    closest_numbers = []
    for number, range_list in zip(state, [POS_RANGE, VEL_RANGE]):
        closest_number = min(range_list, key=lambda x: abs(x - number))
        closest_numbers.append(closest_number)
    return closest_numbers

In [ ]:
def value_iteration(env, state_space, gamma=0.99, theta=0.0001):
    # intialize env
    state = env.reset()[0]

    # total iterations
    total_iterations = 0

    # Initialize value function
    value_function = initialize_value_function(state_space)


    # Perform value iteration until convergence
    while True:
        total_iterations += 1
        delta = 0
        # Iterate over each state in the state space
        for state in state_space:
            old_value = value_function[tuple(state)]
            # Calculate the value for the current state using the Bellman equation
            new_value = 0
            for action in range(env.action_space.n):
                # Set the environment state to the current state
                env.state = np.array(state)
                print(env.state())
                # Take action in the environment
                next_state, reward, done, trunc, _ = env.step(action)
                print(reward)
                next_state = get_state_bin(next_state)
                print(value_function)
                # Update the value function based on the Bellman equation
                new_value += (reward + gamma * value_function[tuple(next_state)]) / env.action_space.n
                env.reset()
            # Update the value function for the current state
            value_function[tuple(state)] = new_value
            # Update the maximum change in value function
            delta = max(delta, abs(new_value - old_value))
            print("Iteration:", total_iterations, "State:", state, "Value:", value_function[tuple(state)])
        # Check for convergence
        if delta < theta and  total_iterations > 50:
            break
    return value_function

state_space = get_stare_space()
# Perform value iteration
# value_function = value_iteration(env, state_space)

# Example value for a state
position_index = 0
velocity_index = 0
# print("Value for state (position_index={}, velocity_index={}): {}".format(position_index, velocity_index, value_function[position_index, velocity_index]))

In [5]:
# Value Iteration
verbose = False

# Define the number of bins for each parameter
num_bins = 5

# Define the ranges for each parameter
cart_position_range = [-0.5, 0.5]
cart_velocity_range = [-0.5, 0.5]  # If value is infinity, set to 10
pole_angle_range = [-0.2, 0.2]
pole_angular_velocity_range = [-0.5, 0.5]  # If value is infinity, set to 10


# Function to create the state space
def create_state_space():
    # Create lists to store the discretized states
    state_space = []

    # Create bins for each parameter
    for cart_pos in range(num_bins):
        for cart_vel in range(num_bins):
            for pole_angle in range(num_bins):
                for pole_ang_vel in range(num_bins):
                    # Calculate the value for each parameter
                    cart_pos_value = cart_position_range[0] + cart_pos * (cart_position_range[1] - cart_position_range[0]) / (num_bins - 1)
                    cart_vel_value = cart_velocity_range[0] + cart_vel * (cart_velocity_range[1] - cart_velocity_range[0]) / (num_bins - 1)
                    pole_angle_value = pole_angle_range[0] + pole_angle * (pole_angle_range[1] - pole_angle_range[0]) / (num_bins - 1)
                    pole_ang_vel_value = pole_angular_velocity_range[0] + pole_ang_vel * (pole_angular_velocity_range[1] - pole_angular_velocity_range[0]) / (num_bins - 1)

                    # Add the discretized state to the state space list
                    state_space.append([cart_pos_value, cart_vel_value, pole_angle_value, pole_ang_vel_value])
    return state_space


# Function to initialize the value function for each state to zero
def initialize_value_function(state_space):
    value_function = {}
    for state in state_space:
        value_function[tuple(state)] = 0
    return value_function


def get_state_bin(state):
    # get the ranges of each of the parameters divided by bins
    ranges = []
    ranges.append(np.linspace(cart_position_range[0], cart_position_range[1], num_bins))
    ranges.append(np.linspace(cart_velocity_range[0], cart_velocity_range[1], num_bins))
    ranges.append(np.linspace(pole_angle_range[0], pole_angle_range[1], num_bins))
    ranges.append(np.linspace(pole_angular_velocity_range[0], pole_angular_velocity_range[1], num_bins))

    closest_numbers = []
    for number, range_list in zip(state, ranges):
        closest_number = min(range_list, key=lambda x: abs(x - number))
        closest_numbers.append(closest_number)
    return closest_numbers


# Function to perform value iteration
def value_iteration(env, state_space, gamma=0.99, theta=0.00001):
    # intialize env
    state = env.reset()[0]

    # total iterations
    total_iterations = 0

    # Initialize value function
    value_function = initialize_value_function(state_space)

    # Perform value iteration until convergence
    while True:
        total_iterations += 1
        delta = 0
        # Iterate over each state in the state space
        for state in state_space:
            old_value = value_function[tuple(state)]
            # Calculate the value for the current state using the Bellman equation
            new_value = 0
            for action in range(env.action_space.n):
                # Set the environment state to the current state
                env.state = np.array(state)
                # Take action in the environment
                next_state, reward, done, trunc, _ = env.step(action)
                print(reward)
                next_state = get_state_bin(next_state)
                # Update the value function based on the Bellman equation
                new_value += (reward + gamma * value_function[tuple(next_state)]) / env.action_space.n
                if done:
                    env.reset()
            # Update the value function for the current state
            value_function[tuple(state)] = new_value
            # Update the maximum change in value function
            delta = max(delta, abs(new_value - old_value))
            if verbose:
                print("Iteration:", total_iterations, "State:", state, "Value:", value_function[tuple(state)])
        # Check for convergence
        if delta < theta and  total_iterations > 100:
            break
    return value_function

# gamma
gamma = 0.99
theta = 0.0001

# Discretize the state space
state_space = create_state_space()

# Perform value iteration
V = value_iteration(env, state_space, gamma=gamma, theta=theta)

# Print the value function
for state, value in V.items():
    print("State:", state, "Value:", value)

# Extract and return the optimal policy
# policy = np.zeros(state_space, dtype=int)
# for state in range(state_space.shape[0]):
#     action_values = [sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[state][action]) for action in range(env.action_space.n)]
#     policy[state] = np.argmax(action_values)


# # plot state values
# size=(8,8)
# Plots.values_heat_map(V, "Frozen Lake\nValue Iteration State Values", size)

# # plot policy
# fl_actions = {0: "←", 1: "↓", 2: "→", 3: "↑"}
# fl_map_size=(8,8)
# title="FL Mapped Policy\nArrows represent best action"
# val_max, policy_map = Plots.get_policy_map(policy, V, fl_actions, fl_map_size)
# Plots.plot_policy(val_max, policy_map, fl_map_size, title)

# # Clip trailing zeros in case convergence is reached before max iterations
# # This is likely when setting the n_iters parameter
# max_value_per_iter = np.trim_zeros(np.mean(V_track, axis=1), 'b')
# Plots.v_iters_plot(max_value_per_iter, "Frozen Lake\nMean Value v Iterations")

C:\Users\mccar\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


-1.0


KeyError: (0.5, 0.5, 0.0)

In [1]:
# Value Iteration
verbose = False

# Define the number of bins for each parameter
num_bins = 5

# Define the ranges for each parameter
cart_position_range = [-0.5, 0.5]
cart_velocity_range = [-0.5, 0.5]  # If value is infinity, set to 10
pole_angle_range = [-0.2, 0.2]
pole_angular_velocity_range = [-0.5, 0.5]  # If value is infinity, set to 10


# Function to create the state space
def create_state_space():
    # Create lists to store the discretized states
    state_space = []

    # Create bins for each parameter
    for cart_pos in range(num_bins):
        for cart_vel in range(num_bins):
            for pole_angle in range(num_bins):
                for pole_ang_vel in range(num_bins):
                    # Calculate the value for each parameter
                    cart_pos_value = cart_position_range[0] + cart_pos * (cart_position_range[1] - cart_position_range[0]) / (num_bins - 1)
                    cart_vel_value = cart_velocity_range[0] + cart_vel * (cart_velocity_range[1] - cart_velocity_range[0]) / (num_bins - 1)
                    pole_angle_value = pole_angle_range[0] + pole_angle * (pole_angle_range[1] - pole_angle_range[0]) / (num_bins - 1)
                    pole_ang_vel_value = pole_angular_velocity_range[0] + pole_ang_vel * (pole_angular_velocity_range[1] - pole_angular_velocity_range[0]) / (num_bins - 1)

                    # Add the discretized state to the state space list
                    state_space.append([cart_pos_value, cart_vel_value, pole_angle_value, pole_ang_vel_value])
    return state_space


# Function to initialize the value function for each state to zero
def initialize_value_function(state_space):
    value_function = {}
    for state in state_space:
        value_function[tuple(state)] = 0
    return value_function


def get_state_bin(state):
    # get the ranges of each of the parameters divided by bins
    ranges = []
    ranges.append(np.linspace(cart_position_range[0], cart_position_range[1], num_bins))
    ranges.append(np.linspace(cart_velocity_range[0], cart_velocity_range[1], num_bins))
    ranges.append(np.linspace(pole_angle_range[0], pole_angle_range[1], num_bins))
    ranges.append(np.linspace(pole_angular_velocity_range[0], pole_angular_velocity_range[1], num_bins))

    closest_numbers = []
    for number, range_list in zip(state, ranges):
        closest_number = min(range_list, key=lambda x: abs(x - number))
        closest_numbers.append(closest_number)
    return closest_numbers


# Function to perform value iteration
def value_iteration(env, state_space, gamma=0.99, theta=0.00001):
    # intialize env
    state = env.reset()[0]

    # total iterations
    total_iterations = 0

    # Initialize value function
    value_function = initialize_value_function(state_space)

    # Perform value iteration until convergence
    while True:
        total_iterations += 1
        delta = 0
        # Iterate over each state in the state space
        for state in state_space:
            old_value = value_function[tuple(state)]
            # Calculate the value for the current state using the Bellman equation
            new_value = 0
            for action in range(env.action_space.n):
                # Set the environment state to the current state
                env.state = np.array(state)
                # Take action in the environment
                next_state, reward, done, trunc, _ = env.step(action)
                print(reward)
                next_state = get_state_bin(next_state)
                # Update the value function based on the Bellman equation
                new_value += (reward + gamma * value_function[tuple(next_state)]) / env.action_space.n
                if done:
                    env.reset()
            # Update the value function for the current state
            value_function[tuple(state)] = new_value
            # Update the maximum change in value function
            delta = max(delta, abs(new_value - old_value))
            if verbose:
                print("Iteration:", total_iterations, "State:", state, "Value:", value_function[tuple(state)])
        # Check for convergence
        if delta < theta and  total_iterations > 100:
            break
    return value_function

# gamma
gamma = 0.99
theta = 0.0001

# Discretize the state space
state_space = create_state_space()

# Perform value iteration
V = value_iteration(env, state_space, gamma=gamma, theta=theta)

# Print the value function
for state, value in V.items():
    print("State:", state, "Value:", value)

# Extract and return the optimal policy
# policy = np.zeros(state_space, dtype=int)
# for state in range(state_space.shape[0]):
#     action_values = [sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[state][action]) for action in range(env.action_space.n)]
#     policy[state] = np.argmax(action_values)


# # plot state values
# size=(8,8)
# Plots.values_heat_map(V, "Frozen Lake\nValue Iteration State Values", size)

# # plot policy
# fl_actions = {0: "←", 1: "↓", 2: "→", 3: "↑"}
# fl_map_size=(8,8)
# title="FL Mapped Policy\nArrows represent best action"
# val_max, policy_map = Plots.get_policy_map(policy, V, fl_actions, fl_map_size)
# Plots.plot_policy(val_max, policy_map, fl_map_size, title)

# # Clip trailing zeros in case convergence is reached before max iterations
# # This is likely when setting the n_iters parameter
# max_value_per_iter = np.trim_zeros(np.mean(V_track, axis=1), 'b')
# Plots.v_iters_plot(max_value_per_iter, "Frozen Lake\nMean Value v Iterations")

NameError: name 'env' is not defined

In [ ]:
# Policy Iteration
verbose = True
found_it = False
found_it_iter = 0

# Discretize the state space
def discretize_state(observation, bins):
    state = []
    for i in range(len(observation)):
        state.append(np.digitize(observation[i], bins[i]))
    return tuple(state)

def policy_evaluation(env, policy, gamma=0.99, epsilon=1e-6):
    V = np.zeros(env.observation_space.n)
    while True:
        delta = 0
        for state in range(env.observation_space.n):
            v = V[state]
            action = policy[state]
            V[state] = sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[state][action])
            delta = max(delta, np.abs(v - V[state]))
        if delta < epsilon:
            break
    return V

def policy_improvement(env, V, gamma=0.99):
    policy = np.zeros(env.observation_space.n, dtype=int)
    for state in range(env.observation_space.n):
        action_values = [sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[state][action]) for action in range(env.action_space.n)]
        policy[state] = np.argmax(action_values)
    return policy

def policy_iteration(env, max_iterations=100, gamma=0.99):
    rewards_per_episode = []
    steps_per_episode = []
    
    policy = np.random.randint(env.action_space.n, size=env.observation_space.n)
    for epi in range(max_iterations):
        V = policy_evaluation(env, policy, gamma)
        new_policy = policy_improvement(env, V, gamma)
        
        # Track rewards and steps per episode
        episode_reward = 0
        episode_steps = 0
        
        state = env.reset()[0]
        while True:
            action = new_policy[state]
            next_state, reward, done, trunc, _ = env.step(action)
            episode_reward += reward
            episode_steps += 1

            if reward > 0:
                if not found_it:
                    found_it = True
                    found_it_iter = epi
                print("Found IT!!!!")

            if done or trunc:
                rewards_per_episode.append(episode_reward)
                steps_per_episode.append(episode_steps)
                break
            state = next_state
        
        if verbose:
            print(f"Episode {epi+1} - Reward: {episode_reward}, Steps: {episode_steps}")
        
        if np.array_equal(new_policy, policy) and epi > 100:
            break
        policy = new_policy
    return policy, V, rewards_by_episode, steps_by_episode, found_it_iter


# Solve the Frozen Lake problem using Policy Iteration
policy, V, rewards_by_episode, steps_by_episode, found_it_iter = policy_iteration(env)


# moving avgs
reward_moving_avg_10 = np.convolve(rewards_by_episode, np.ones(10)/10, mode='valid')
reward_moving_avg_50 = np.convolve(rewards_by_episode, np.ones(50)/50, mode='valid')
reward_moving_avg_100 = np.convolve(rewards_by_episode, np.ones(100)/100, mode='valid')
reward_moving_avg_500 = np.convolve(rewards_by_episode, np.ones(500)/500, mode='valid')
reward_moving_avg_1000 = np.convolve(rewards_by_episode, np.ones(1000)/1000, mode='valid')
steps_moving_avg_10 = np.convolve(steps_by_episode, np.ones(10)/10, mode='valid')
steps_moving_avg_50 = np.convolve(steps_by_episode, np.ones(50)/50, mode='valid')
steps_moving_avg_100 = np.convolve(steps_by_episode, np.ones(100)/100, mode='valid')
steps_moving_avg_500 = np.convolve(steps_by_episode, np.ones(500)/500, mode='valid')
steps_moving_avg_1000 = np.convolve(steps_by_episode, np.ones(1000)/1000, mode='valid')


# # Extract and return the optimal policy
# policy = np.zeros(env.observation_space.n, dtype=int)
# for state in range(env.observation_space.n):
#     action_values = [sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[state][action]) for action in range(env.action_space.n)]
#     policy[state] = np.argmax(action_values)

# Plot the convergence
fig, ax1 = plt.subplots(figsize=(10, 6))
color = 'tab:red'
ax1.set_xlabel('Episode')
ax1.set_ylabel('Steps per Episode', color=color)
# ax1.plot(steps_moving_avg_1000, label='Steps (MA-1000)', color=color)
ax1.plot(steps_moving_avg_500, label='Steps (MA-500)', color=color)
ax1.plot(steps_moving_avg_100, label='Steps (MA-100)', color=color, linestyle='dashed')
# ax1.plot(steps_moving_avg_50, label='Steps (MA-50)', color=color, linestyle='dashed')
# ax1.plot(steps_moving_avg_10, label='Steps (MA-10)', color=color, linestyle='dotted')
ax1.tick_params(axis='y', labelcolor=color)
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Reward per Episode', color=color)
# ax2.plot(reward_moving_avg_1000, label='Reward (MA-1000)', color=color)
ax2.plot(reward_moving_avg_500, label='Reward (MA-500)', color=color)
ax2.plot(reward_moving_avg_100, label='Reward (MA-100)', color=color, linestyle='dashed')
# ax2.plot(reward_moving_avg_50, label='Reward (MA-50)', color=color, linestyle='dashed')
# ax2.plot(reward_moving_avg_10, label='Reward (MA-10)', color=color, linestyle='dotted')
ax2.tick_params(axis='y', labelcolor=color)
fig.tight_layout()
plt.title('Convergence of Value Iteration Moving Averages (Steps and Reward per Episode)')
plt.show()

# Print the optimal policy
# print("Optimal Policy:")
# print(policy.reshape((8, 8)))
print("Found it in iteration: ", found_it_iter)

# plot state values
size=(8,8)
Plots.values_heat_map(V, "Frozen Lake\nValue Iteration State Values", size)

# plot policy
fl_actions = {0: "←", 1: "↓", 2: "→", 3: "↑"}
fl_map_size=(8,8)
title="FL Mapped Policy\nArrows represent best action"
val_max, policy_map = Plots.get_policy_map(policy, V, fl_actions, fl_map_size)
Plots.plot_policy(val_max, policy_map, fl_map_size, title)

print(np.array(print_env))


# Print the optimal policy
print("Optimal Policy:")
print(policy.reshape((8, 8)))

# Print the value function
print("\nValue Function:")
print(V.reshape((8, 8)))

AttributeError: 'Box' object has no attribute 'n'